# Embedding Similarity Analysis
This notebook loads embeddings and transcript data, then allows interactive similarity calculations between any two time points.

In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 1. Configuration - Update Paths

In [35]:
# UPDATE THESE PATHS
EMBEDDINGS_NPY = "./second_level_embeddings_first_debate_with_transcript.npy"
#EMBEDDINGS_NPY = "./video_embeddings_per_second.npy"
TRANSCRIPT_CSV = "/storage/home/saichandc/qwen/transcript_by_second_first_debate.csv"

## 2. Load Embeddings

In [36]:
# Load embeddings
embeds = np.load(EMBEDDINGS_NPY)

print(f"Loaded embeddings with shape: {embeds.shape}")
print(f"\n✓ Total number of embeddings: {embeds.shape[0]}")
print(f"✓ Embedding dimension: {embeds.shape[1]}")

Loaded embeddings with shape: (2898, 2048)

✓ Total number of embeddings: 2898
✓ Embedding dimension: 2048

## 3. Load Transcript CSV

In [37]:
# Load transcript
df = pd.read_csv(TRANSCRIPT_CSV)

print(f"Loaded transcript CSV with shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Loaded transcript CSV with shape: (2898, 2)
Columns: ['second', 'transcript']

First few rows:

,second,transcript
0,0,Good
1,1,evening from
2,2,the Health Education Campus
3,3,of
4,4,Case Western Reserve


## 4. Auto-detect Column Names

In [38]:
# Auto-detect time and text columns
time_col = None
text_col = None

for col in df.columns:
    col_lower = col.lower()
    if col_lower in ['second', 'sec', 'time', 'seconds']:
        time_col = col
    if col_lower in ['text', 'transcript', 'content', 'speech']:
        text_col = col

if time_col is None:
    time_col = df.columns[0]
if text_col is None:
    text_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]

print(f"Using time column: '{time_col}'")
print(f"Using text column: '{text_col}'")

Using time column: 'second'
Using text column: 'transcript'

## 5. Helper Functions

In [39]:
def get_text_for_second(sec: int) -> str:
    """Get text for a specific second from transcript"""
    row = df[df[time_col] == sec]
    
    if len(row) == 0:
        return f"[No text found for second {sec}]"
    
    text = row[text_col].iloc[0]
    return str(text) if pd.notna(text) else "[Empty]"

def calculate_similarity(sec1: int, sec2: int) -> float:
    """Calculate cosine similarity between embeddings at two time points"""
    if sec1 >= len(embeds) or sec2 >= len(embeds):
        raise ValueError(f"Time index out of range. Valid range: 0-{len(embeds)-1}")
    
    # Reshape for sklearn
    emb1 = embeds[sec1].reshape(1, -1)
    emb2 = embeds[sec2].reshape(1, -1)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(emb1, emb2)[0, 0]
    return similarity

def compare_seconds(sec1: int, sec2: int):
    """Compare two seconds and display similarity + text"""
    if sec1 < 0 or sec1 >= len(embeds):
        print(f"ERROR: First second must be between 0 and {len(embeds)-1}")
        return
    if sec2 < 0 or sec2 >= len(embeds):
        print(f"ERROR: Second second must be between 0 and {len(embeds)-1}")
        return
    
    # Calculate similarity
    similarity = calculate_similarity(sec1, sec2)
    
    # Get texts
    text1 = get_text_for_second(sec1)
    text2 = get_text_for_second(sec2)
    
    # Display results
    print("=" * 70)
    print(f"SIMILARITY: Second {sec1} vs Second {sec2}")
    print("=" * 70)
    print(f"\n📊 Cosine Similarity: {similarity:.4f}")
    print(f"\n📝 Text at second {sec1}:")
    print(f"   {text1}")
    print(f"\n📝 Text at second {sec2}:")
    print(f"   {text2}")
    print("=" * 70)

print("✓ Helper functions defined")

✓ Helper functions defined

## 6. Compare Two Time Points
Run this cell and modify `sec1` and `sec2` to compare different time points

In [46]:
# MODIFY THESE VALUES to compare different time points
sec1 = 1  # First second
sec2 = 1446  # Second second

compare_seconds(sec1, sec2)

SIMILARITY: Second 1 vs Second 1446

📊 Cosine Similarity: 0.8126

📝 Text at second 1:
   evening from

📝 Text at second 1446:
   & Johnson,

## 7. Quick Lookups
View text at a specific second

In [29]:
# View text at a specific second
sec = 2897  # Change this

print(f"Second {sec}: {get_text_for_second(sec)}")

Second 2897: about here.

## 8. Find Most Similar Second to a Given Time
Find which other seconds are most similar to a target second

In [47]:
def find_most_similar(target_sec: int, top_n: int = 5, exclude_self: bool = True):
    """Find the most similar seconds to a target second"""
    if target_sec >= len(embeds):
        print(f"ERROR: Second must be between 0 and {len(embeds)-1}")
        return
    
    # Get target embedding
    target_emb = embeds[target_sec].reshape(1, -1)
    
    # Calculate similarity with all embeddings
    similarities = cosine_similarity(target_emb, embeds)[0]
    
    # Get top N indices
    if exclude_self:
        similarities[target_sec] = -1  # Exclude self
    
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    
    # Display results
    print("=" * 70)
    print(f"Top {top_n} most similar seconds to second {target_sec}")
    print("=" * 70)
    print(f"\nTarget (second {target_sec}): {get_text_for_second(target_sec)}")
    print("\n" + "=" * 70)
    
    for i, idx in enumerate(top_indices, 1):
        sim = similarities[idx]
        text = get_text_for_second(idx)
        print(f"\n{i}. Second {idx} (similarity: {sim:.4f})")
        print(f"   {text}")
    print("\n" + "=" * 70)

# Example usage
target_second = 1446  # Change this
find_most_similar(target_second, top_n=10)

Top 10 most similar seconds to second 1446

Target (second 1446): & Johnson,


1. Second 1445 (similarity: 0.9916)
   Moderna, Johnson

2. Second 1821 (similarity: 0.9913)
   Biden.

3. Second 1444 (similarity: 0.9912)
   speak to. We have great –

4. Second 1451 (similarity: 0.9904)
   political, because

5. Second 1470 (similarity: 0.9898)
   the answer

6. Second 2689 (similarity: 0.9897)
   back manufacturing.

7. Second 2379 (similarity: 0.9892)
   When? But let me just tell

8. Second 1442 (similarity: 0.9892)
   to all

9. Second 1450 (similarity: 0.9890)
   Become very

10. Second 2015 (similarity: 0.9889)
   to


In [48]:
def find_most_dissimilar(target_sec: int, top_n: int = 5, exclude_self: bool = True):
    """Find the most dissimilar seconds to a target second"""
    if target_sec >= len(embeds):
        print(f"ERROR: Second must be between 0 and {len(embeds)-1}")
        return
    
    # Get target embedding
    target_emb = embeds[target_sec].reshape(1, -1)
    
    # Calculate similarity with all embeddings
    similarities = cosine_similarity(target_emb, embeds)[0]
    
    # Exclude self if requested
    if exclude_self:
        similarities[target_sec] = 2  # Set to high value to exclude (similarity range is -1 to 1)
    
    # Get bottom N indices (most dissimilar = lowest similarity)
    bottom_indices = np.argsort(similarities)[:top_n]
    
    # Display results
    print("=" * 70)
    print(f"Top {top_n} most DISSIMILAR seconds to second {target_sec}")
    print("=" * 70)
    print(f"\nTarget (second {target_sec}): {get_text_for_second(target_sec)}")
    print("\n" + "=" * 70)
    
    for i, idx in enumerate(bottom_indices, 1):
        sim = similarities[idx]
        text = get_text_for_second(idx)
        print(f"\n{i}. Second {idx} (similarity: {sim:.4f})")
        print(f"   {text}")
    print("\n" + "=" * 70)

# Example usage
target_second = 1  # Change this
find_most_dissimilar(target_second, top_n=10)

Top 10 most DISSIMILAR seconds to second 1

Target (second 1): evening from


1. Second 816 (similarity: 0.5613)
   it. Let him – let him – there is no manifesto.

2. Second 1247 (similarity: 0.5693)
   minute. You didn't

3. Second 1923 (similarity: 0.5768)
   one of the last

4. Second 2771 (similarity: 0.5824)
   [Empty]

5. Second 201 (similarity: 0.5887)
   candidate. As good as anybody

6. Second 2797 (similarity: 0.5918)
   him $183

7. Second 167 (similarity: 0.5939)
   have consequences.

8. Second 2516 (similarity: 0.5954)
   plan would

9. Second 2784 (similarity: 0.6003)
   .5 million.

10. Second 916 (similarity: 0.6036)
   I got you. Premiums


## 9. Summary Statistics

In [43]:
print("=" * 70)
print("DATASET SUMMARY")
print("=" * 70)
print(f"Total embeddings: {embeds.shape[0]}")
print(f"Embedding dimension: {embeds.shape[1]}")
print(f"Transcript entries: {len(df)}")
print(f"Valid time range: 0 to {embeds.shape[0]-1} seconds")
print("=" * 70)

DATASET SUMMARY
Total embeddings: 2898
Embedding dimension: 2048
Transcript entries: 2898
Valid time range: 0 to 2897 seconds

In [6]:
"""
Compute Local Surprise using Squared Euclidean Distance
Window = 1 (compare each second to immediately previous second)
"""

import numpy as np
import pandas as pd

# ----------------------------
# Configuration
# ----------------------------
EMBEDDINGS_NPY = "/storage/home/saichandc/qwen/second_level_embeddings_first_debate_text_only.npy"
TRANSCRIPT_CSV = "/storage/home/saichandc/qwen/transcript_by_second_first_debate.csv"
OUTPUT_CSV = "/storage/home/saichandc/qwen/surprise_scores_euclidean.csv"

CONTEXT_WINDOW = 1  # Compare to immediately previous second

# ----------------------------
# Compute Local Surprise
# ----------------------------
def compute_local_surprise_l2_squared(embeds: np.ndarray, window: int = 1) -> np.ndarray:
    """
    Local surprise using L2 norm squared
    
    Surprise(t) = ||embedding(t) - mean(context)||^2
    
    With window=1, this is just the squared distance from previous second:
    Surprise(t) = ||embedding(t) - embedding(t-1)||^2
    """
    n = len(embeds)
    surprise = np.zeros(n)
    
    for i in range(n):
        if i == 0:
            surprise[i] = 0.0  # First second has no previous
            continue
        
        # Get context: previous 'window' seconds
        start_idx = max(0, i - window)
        context = embeds[start_idx:i]
        
        # Average of context (with window=1, this is just the previous embedding)
        context_mean = context.mean(axis=0)
        
        # Squared L2 norm of difference
        diff = embeds[i] - context_mean
        surprise[i] = np.sum(diff ** 2)  # ||diff||^2
    
    return surprise

# ----------------------------
# Main
# ----------------------------
def main():
    print("Computing local surprise using squared Euclidean distance...")
    print(f"Context window: {CONTEXT_WINDOW} second (immediate predecessor)\n")
    
    # Load embeddings
    embeds = np.load(EMBEDDINGS_NPY)
    print(f"Loaded embeddings: {embeds.shape}")
    print(f"Embedding dimension: {embeds.shape[1]}")
    
    # Load transcript
    df = pd.read_csv(TRANSCRIPT_CSV)
    
    # Auto-detect columns
    time_col = next((col for col in df.columns if col.lower() in ['second', 'sec', 'time', 'seconds']), df.columns[0])
    text_col = next((col for col in df.columns if col.lower() in ['text', 'transcript', 'content', 'speech']), df.columns[1])
    
    # Compute surprise
    print(f"\nComputing surprise (comparing each second to previous second)...")
    surprise = compute_local_surprise_l2_squared(embeds, window=CONTEXT_WINDOW)
    
    # Create results
    results = pd.DataFrame({
        'second': df[time_col][:len(embeds)],
        'text': df[text_col][:len(embeds)],
        'surprise_raw': surprise
    })
    
    # Save
    results.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✓ Saved to: {OUTPUT_CSV}")
    
    # Statistics
    print(f"\nSurprise Statistics (raw):")
    print(f"  Mean: {surprise.mean():.4f}")
    print(f"  Std:  {surprise.std():.4f}")
    print(f"  Min:  {surprise.min():.4f}")
    print(f"  Max:  {surprise.max():.4f}")
    
    # Top 10 most surprising (biggest jumps from previous second)
    print(f"\n{'='*70}")
    print("TOP 10 MOST SURPRISING MOMENTS (Biggest change from previous second)")
    print('='*70)
    top_10 = results.nlargest(10, 'surprise_raw')
    for _, row in top_10.iterrows():
        sec = int(row['second'])
        print(f"\nSecond {sec} (Surprise: {row['surprise_raw']:.4f})")
        
        # Show current and previous text
        if sec > 0:
            # Get previous text safely
            prev_row = results[results['second'] == sec - 1]
            if len(prev_row) > 0:
                prev_text = prev_row['text'].values[0]
                # Handle NaN or non-string values
                if pd.isna(prev_text):
                    prev_text = "[Empty]"
                else:
                    prev_text = str(prev_text)
            else:
                prev_text = "[N/A]"
            
            # Get current text safely
            curr_text = row['text']
            if pd.isna(curr_text):
                curr_text = "[Empty]"
            else:
                curr_text = str(curr_text)
            
            print(f"  Previous ({sec-1}): {prev_text[:100]}...")
            print(f"  Current  ({sec}): {curr_text[:100]}...")
        else:
            curr_text = row['text']
            if pd.isna(curr_text):
                curr_text = "[Empty]"
            else:
                curr_text = str(curr_text)
            print(f"  {curr_text[:100]}...")
    
    return results

if __name__ == "__main__":
    results = main()

Computing local surprise using squared Euclidean distance...
Context window: 1 second (immediate predecessor)

Loaded embeddings: (2898, 2048)
Embedding dimension: 2048

Computing surprise (comparing each second to previous second)...

✓ Saved to: /storage/home/saichandc/qwen/surprise_scores_euclidean.csv

Surprise Statistics (raw):
  Mean: 1586.0371
  Std:  1198.7875
  Min:  0.0000
  Max:  14049.2744

TOP 10 MOST SURPRISING MOMENTS (Biggest change from previous second)

Second 968 (Surprise: 14049.2744)
  Previous (967): No, no, no, no. But in terms of the court, Justice Barrett,...
  Current  (968): there...

Second 2539 (Surprise: 12913.5059)
  Previous (2538): [Empty]...
  Current  (2539): it would be about putting out the And number four, it would be about making sure that...

Second 2666 (Surprise: 10644.5020)
  Previous (2665): [Empty]...
  Current  (2666): Wait, wait. Is it fair to say he blew...

Second 2540 (Surprise: 10206.5137)
  Previous (2539): it would be about putting o

In [7]:
"""
Compute Speed: Average Euclidean distance traveled between consecutive embeddings
Time base = 1 second (so distance values = speed at each timestep)
"""

import numpy as np
import pandas as pd

# ----------------------------
# Configuration
# ----------------------------
EMBEDDINGS_NPY = "/storage/home/saichandc/qwen/second_level_embeddings_first_debate_text_only.npy"
TRANSCRIPT_CSV = "/storage/home/saichandc/qwen/transcript_by_second_first_debate.csv"
OUTPUT_CSV = "/storage/home/saichandc/qwen/speed_scores.csv"

# ----------------------------
# Compute Distance and Speed
# ----------------------------
def compute_distance_and_speed(embeds: np.ndarray):
    """
    Compute Euclidean distance between consecutive embeddings
    
    With time base = 1 second:
    - distance(t) = ||x_{t+1} - x_t|| (units: embedding distance / second)
    - speed(t) = distance(t) / 1 second = distance(t) (numerically same)
    - average_speed = mean(speed(t)) = mean(distance(t))
    
    Returns:
        distances: array of distances for each timestep (= instantaneous speed)
        avg_speed: average speed (scalar)
    """
    n = len(embeds)
    distances = np.zeros(n)
    
    for i in range(n - 1):
        # Euclidean distance (NOT squared)
        diff = embeds[i+1] - embeds[i]
        distances[i] = np.sqrt(np.sum(diff ** 2))  # ||x_{t+1} - x_t||
    
    # Last timestep has no next point
    distances[-1] = 0.0
    
    # Average speed = mean of all instantaneous speeds
    avg_speed = distances[:-1].mean() if n > 1 else 0.0
    
    return distances, avg_speed

# ----------------------------
# Main
# ----------------------------
def main():
    print("Computing semantic speed from text embeddings...")
    print("Time base = 1 second")
    print("  distance(t) = ||x_{t+1} - x_t|| (embedding distance units)")
    print("  speed(t) = distance(t) / 1 sec (embedding distance/second)")
    print("  → Numerically identical, but different units\n")
    
    # Load embeddings
    embeds = np.load(EMBEDDINGS_NPY)
    print(f"Loaded embeddings: {embeds.shape}")
    print(f"Embedding dimension: {embeds.shape[1]}")
    
    # Load transcript
    df = pd.read_csv(TRANSCRIPT_CSV)
    
    # Auto-detect columns
    time_col = next((col for col in df.columns if col.lower() in ['second', 'sec', 'time', 'seconds']), df.columns[0])
    text_col = next((col for col in df.columns if col.lower() in ['text', 'transcript', 'content', 'speech']), df.columns[1])
    
    # Compute distances and speed
    print(f"\nComputing distances/speeds between consecutive seconds...")
    distances, avg_speed = compute_distance_and_speed(embeds)
    
    print(f"\n{'='*70}")
    print(f"AVERAGE SPEED")
    print(f"{'='*70}")
    print(f"Average speed = mean(distance(t)) = {avg_speed:.6f} units/second")
    print(f"Total distance traveled = {distances.sum():.6f} units")
    print(f"{'='*70}")
    
    # Create results
    # Note: distance and speed are numerically the same with time base = 1
    results = pd.DataFrame({
        'second': df[time_col][:len(embeds)],
        'text': df[text_col][:len(embeds)],
        'distance': distances,  # distance traveled
        'speed': distances      # speed = distance/1sec (numerically same)
    })
    
    # Save
    results.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✓ Saved to: {OUTPUT_CSV}")
    
    # Statistics
    print(f"\nSpeed/Distance Statistics:")
    print(f"  Mean:   {distances.mean():.6f} units/sec")
    print(f"  Std:    {distances.std():.6f} units/sec")
    print(f"  Min:    {distances.min():.6f} units/sec")
    print(f"  Max:    {distances.max():.6f} units/sec")
    print(f"  Median: {np.median(distances):.6f} units/sec")
    
    # Top 10 highest speeds (largest semantic jumps)
    print(f"\n{'='*70}")
    print("TOP 10 HIGHEST SEMANTIC SPEEDS (Largest jumps)")
    print('='*70)
    top_10 = results.nlargest(10, 'speed')
    for _, row in top_10.iterrows():
        sec = int(row['second'])
        print(f"\nSecond {sec} → {sec+1} (Speed: {row['speed']:.6f} units/sec)")
        
        # Show current and next text
        curr_text = row['text']
        if pd.isna(curr_text):
            curr_text = "[Empty]"
        else:
            curr_text = str(curr_text)
        
        # Get next text
        if sec < len(results) - 1:
            next_row = results[results['second'] == sec + 1]
            if len(next_row) > 0:
                next_text = next_row['text'].values[0]
                if pd.isna(next_text):
                    next_text = "[Empty]"
                else:
                    next_text = str(next_text)
            else:
                next_text = "[N/A]"
            
            print(f"  From ({sec}): {curr_text[:100]}...")
            print(f"  To   ({sec+1}): {next_text[:100]}...")
        else:
            print(f"  Current ({sec}): {curr_text[:100]}...")
    
    # Summary
    print(f"\n{'='*70}")
    print("SUMMARY")
    print('='*70)
    print(f"Total seconds analyzed: {len(embeds)}")
    print(f"Average semantic speed: {avg_speed:.6f} units/sec")
    print(f"Total semantic distance: {distances.sum():.6f} units")
    print(f"\nNote: With time base = 1 second:")
    print(f"  - distance and speed are numerically identical")
    print(f"  - distance: spatial units in embedding space")
    print(f"  - speed: spatial units per second")
    print('='*70)
    
    return results, avg_speed

if __name__ == "__main__":
    results, speed = main()

Computing semantic speed from text embeddings...
Time base = 1 second
  distance(t) = ||x_{t+1} - x_t|| (embedding distance units)
  speed(t) = distance(t) / 1 sec (embedding distance/second)
  → Numerically identical, but different units

Loaded embeddings: (2898, 2048)
Embedding dimension: 2048

Computing distances/speeds between consecutive seconds...

AVERAGE SPEED
Average speed = mean(distance(t)) = 37.740711 units/second
Total distance traveled = 109334.841009 units

✓ Saved to: /storage/home/saichandc/qwen/speed_scores.csv

Speed/Distance Statistics:
  Mean:   37.727688 units/sec
  Std:    12.753768 units/sec
  Min:    0.000000 units/sec
  Max:    118.529633 units/sec
  Median: 35.260599 units/sec

TOP 10 HIGHEST SEMANTIC SPEEDS (Largest jumps)

Second 967 → 968 (Speed: 118.529633 units/sec)
  From (967): No, no, no, no. But in terms of the court, Justice Barrett,...
  To   (968): there...

Second 2538 → 2539 (Speed: 113.637611 units/sec)
  From (2538): [Empty]...
  To   (2539):